In [ ]:
import time
import gzip
import numpy as np
from tqdm import tqdm
from scipy.stats import qmc

from heston.P1P2Heston import heston_call_price
from heston.QuantlibHeston import heston_option_price 
from heston.CosMethodHeston import ChFHestonModel, COS_Heston_Price
from heston.MonteCarloHeston import GeneratePathsHestonAES, price_option_mc
from BlackScholes.BSprice import implied_volatility

# ----------------------------
# Load reference dataset (for param ranges)
# ----------------------------
file = gzip.GzipFile('heston_data/HestonTrainSet.txt.gz', 'r')
data = np.load(file)
params_ref = data[:, :5]

param_names = ['v0', 'rho', 'sigma', 'theta', 'kappa']
param_ranges = {}

print("\nParameter ranges from reference dataset:")
for i, name in enumerate(param_names):
    col = params_ref[:, i]
    param_ranges[name] = (float(col.min()), float(col.max()))
    print(f"{name}: min = {col.min():.6f}, max = {col.max():.6f}")

# ----------------------------
# Config
# ----------------------------
S0 = 1.0
r = 0.0
q = 0.0

strikes = np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5])  # 11 strikes
maturities = np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0])           # 8 maturities

num_iv = len(strikes) * len(maturities)   # 88
row_len = 5 + num_iv

#mode = 'P1P2'
#mode = 'ql'
mode = 'COS'
#mode = 'MC-AES'
NoOfPaths, NoOfSteps = 100000, 200

print(f"Mode: {mode}")

# ----------------------------
# Latin Hypercube Sampling until target
# ----------------------------
N_target = 20000
IV_list, CALL_list = [], []
batch_size = 5000   # generate in chunks
seed = 42
sampler = qmc.LatinHypercube(d=5, seed=seed)

start_time = time.time()

# convenience bounds
l_bounds = [param_ranges[name][0] for name in param_names]
u_bounds = [param_ranges[name][1] for name in param_names]

while len(IV_list) < N_target:
    lhs_sample = sampler.random(n=batch_size)
    lhs_scaled = qmc.scale(lhs_sample, l_bounds, u_bounds)

    for row_params in lhs_scaled:
        v0, rho, sigma, theta, kappa = row_params

        # --- Simple sanity + Feller condition ---
        if not (-0.999 < rho < 0.999):   # keep |rho|<1
            continue
        if v0 < 0 or theta <= 0 or kappa <= 0 or sigma <= 0:
            continue
        if 2.0 * kappa * theta < sigma**2:   # Feller condition
            continue

        ivs, calls = [], []
        valid = True

        for T in maturities:
            for K in strikes:
                try:
                    if mode == 'P1P2':
                        call = heston_call_price(S0, K, r, T, kappa, theta, sigma, rho, v0)
                    elif mode == 'ql':
                        call = heston_option_price(S0, K, r, q, T, v0, kappa, theta, sigma, rho, option_type="call")
                    elif mode == 'COS':
                        cf = ChFHestonModel(r, T, kappa, sigma, theta, v0, rho)
                        call = COS_Heston_Price(cf, "call", S0, r, T, K)
                    elif mode == 'MC-AES':
                        aes_paths = GeneratePathsHestonAES(NoOfPaths, NoOfSteps, T, r, S0,
                                                           kappa, sigma, rho, theta, v0, seed=42)
                        call = price_option_mc("call", aes_paths["S"], K, r, T)
                    else:
                        raise ValueError(f"Unknown mode: {mode}")

                    if call is None or not np.isfinite(call) or call < 0:
                        valid = False
                        break

                    iv_c = implied_volatility(call, S0, K, T, r, q, option_type="call")
                    if iv_c is None or not np.isfinite(iv_c):
                        valid = False
                        break

                    ivs.append(iv_c)
                    calls.append(call)

                except Exception:
                    valid = False
                    break
            if not valid:
                break

        if valid:
            IV_list.append([v0, rho, sigma, theta, kappa] + ivs)
            CALL_list.append([v0, rho, sigma, theta, kappa] + calls)

            if len(IV_list) >= N_target:
                break

IV = np.array(IV_list, dtype=np.float32)
call_prices = np.array(CALL_list, dtype=np.float32)

print("\nFinal dataset shape (IVs):", IV.shape)
print("Final dataset shape (Call Prices):", call_prices.shape)

# ----------------------------
# Save to .npy
# ----------------------------
np.save(f"heston_data/Heston_{mode}_LHS_Feller_IV_r_{r}_T_K_20k.npy", IV)
np.save(f"heston_data/Heston_{mode}_LHS_Feller_CALL_r_{r}_T_K_20k.npy", call_prices)

end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")



Parameter ranges from reference dataset:
v0: min = 0.000101, max = 0.039992
rho: min = -0.949763, max = -0.100011
sigma: min = 0.010014, max = 0.999971
theta: min = 0.013286, max = 0.200000
kappa: min = 1.017702, max = 9.999741
Mode: COS

Final dataset shape (IVs): (20, 93)
Final dataset shape (Call Prices): (20, 93)
Execution time: 1.50 seconds
